In [1]:
# Native libraries
import os
import math
# Essential Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Preprocessing
from sklearn.preprocessing import MinMaxScaler
# Algorithms
from tslearn.barycenters import dtw_barycenter_averaging
from tslearn.clustering import TimeSeriesKMeans
from sklearn.cluster import KMeans

from sklearn.decomposition import PCA

ModuleNotFoundError: No module named 'tslearn'

In [ ]:
csv = pd.read_csv("monthly_data.csv")

In [ ]:
num = 0
for i in range(270, 271):
    num += list(csv.isna().sum()).count(i)


In [ ]:
num

In [ ]:
target = pd.read_csv("target.csv")

In [ ]:

corrs_dict = {}
for i in csv:
    if csv[i].isna().sum() >= 100:
        continue
    try:
        corr = csv[i].corr(target["sales"])
 
        corrs_dict[i] = corr
    except: 
        continue

In [ ]:
newlist = [x for x in list(corrs_dict.values()) if np.isnan(x) == False ]
sorted(newlist)

In [ ]:
indices = [i for i, x in enumerate(list(corrs_dict.values())) if x >=0.9]

In [ ]:
list(corrs_dict.keys())[indices[1]]

In [ ]:
corrs_dict["TURCP080000GPM"]

In [ ]:
data = csv.iloc(axis=1)[1:100]

In [ ]:
array = data.fillna(method="bfill").fillna(method="ffill").fillna(0.).to_numpy()


In [ ]:
new_array = np.delete(array, 0, 1)


In [ ]:
ms = MeanShift()

In [ ]:
ms.fit(new_array)

In [ ]:
import matplotlib.pyplot as plt
from itertools import cycle
labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)
plt.figure(1)
plt.clf()
colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
for k, col in zip(range(n_clusters_), colors):
    my_members = labels == k
    cluster_center = cluster_centers[k]
    plt.plot(new_array[my_members, 0], new_array[my_members, 1], col + '.')
    plt.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
             markeredgecolor='k', markersize=14)
plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()